In [1]:
import numpy as np
import hail as hl
from hail import methods
import scipy as sp
import pandas as pd
from math import sqrt, pi

In [2]:
def fastSVD(A, m, n, k, q):
    
    # STAGE A
    
    omega = np.random.normal(0, 100, (n, 2*k))
    adjointA = A.getH()
    AA = A @ adjointA
    AOmega = A @ omega

    Y = AOmega
    for i in range(0, q):
        Y = AA @ Y

    (Q, R) = np.linalg.qr(Y)

    # STAGE B

    adjointQ = Q.getH()
    B = adjointQ @ A
    (tildeU, S, V) = np.linalg.svd(B)
    U = Q @ tildeU
    
    return U, S, V, Q, adjointQ

In [39]:
# Error bound formulas
# pass in singular values, m, n, k, p

# ||A - QQ*A|| < [1 + 9 sqrt(k + p) sqrt(min(m,n))] * k + 1st largest singular value of A
def errorBound1(S, m, n, k, p):
    singularVal = S[k]
    return (1 + 9 * sqrt(k + p) * sqrt(min(m,n))) * singularVal

# A Posteriori Error Estimation
def aPosterioriError(S, m, n, k, Q, adjointQ, A):
    r = 100
    diff = A - Q @ adjointQ @ A
    max = 0
    for i in range(0,r):
        norm = np.linalg.norm(diff @ np.random.normal(0, 1, (n, 1)), 2)
        if max < norm:
            max = norm
    return max

# Blanczos paper error bound 4.3
def blanczosError(U, S, V, m, n, k, q, A, k1th_sing_val):
    norm_diff = np.linalg.norm(A.transpose() - U @ S @ V.transpose())
    bound = 100 * l * (((m-k)/l) ** (1/(4*q + 2))) * k1th_sing_val
    return norm_diff <= bound

In [4]:
# Run on random data

def makeRandomData():
    m = np.random.randint(50, 300)
    n = np.random.randint(50, 300)
    A = np.asmatrix(np.random.rand(m, n))
    k = int((m + n)/4) # this is based on nothing, just picking some k
    q = 2
    return (A, m, n, k, q)

A, m, n, k, q = makeRandomData()
U, S, V, Q, adjointQ = fastSVD(A, m, n, k, q)

# Look at differences:

(u, s, v) = np.linalg.svd(A)
simple_diff = s - S
print("subtracted difference between singular values: ", simple_diff)
print("maximum singular value difference: ", max(simple_diff))

diff = A - Q @ adjointQ @ A
norm_diff = np.linalg.norm(diff, 2)
print("L2 norm error of ||A - QQ*A||: ", norm_diff)

subtracted difference between singular values:  [-2.84217094e-14  1.24344979e-14  7.10542736e-15  3.55271368e-15
  1.77635684e-15 -5.32907052e-15 -1.77635684e-15  3.55271368e-15
 -5.32907052e-15  1.77635684e-15 -1.77635684e-15 -1.77635684e-15
  1.77635684e-15  1.77635684e-15 -5.32907052e-15  1.77635684e-15
  0.00000000e+00 -1.77635684e-15  1.77635684e-15  3.55271368e-15
  3.55271368e-15  5.32907052e-15  5.32907052e-15 -3.55271368e-15
  7.99360578e-15  1.77635684e-15 -7.99360578e-15 -2.66453526e-15
  3.55271368e-15 -6.21724894e-15 -4.44089210e-15  8.88178420e-16
  2.66453526e-15  0.00000000e+00  2.66453526e-15  2.66453526e-15
  8.88178420e-16  0.00000000e+00  2.66453526e-15  8.88178420e-16
 -8.88178420e-16  4.44089210e-15 -4.44089210e-15 -8.88178420e-16
  8.88178420e-16  0.00000000e+00 -4.44089210e-15 -1.77635684e-15
  2.66453526e-15  0.00000000e+00  8.88178420e-16  2.66453526e-15
  1.77635684e-15 -2.66453526e-15  3.55271368e-15  4.44089210e-15
  3.55271368e-15  0.00000000e+00  8.881784

In [5]:
# Define Hardy-Weinberg population normalization function (from John)

def hwe_normalize(call_expr):
    mt = call_expr._indices.source
    mt = mt.select_entries(__gt=call_expr.n_alt_alleles())
    mt = mt.annotate_rows(__AC=hl.agg.sum(mt.__gt),
                          __n_called=hl.agg.count_where(hl.is_defined(mt.__gt)))
    mt = mt.filter_rows((mt.__AC > 0) & (mt.__AC < 2 * mt.__n_called))

    n_variants = mt.count_rows()
    if n_variants == 0:
        raise FatalError("hwe_normalized: found 0 variants after filtering out monomorphic sites.")

    mt = mt.annotate_rows(__mean_gt=mt.__AC / mt.__n_called)
    mt = mt.annotate_rows(
        __hwe_scaled_std_dev=hl.sqrt(mt.__mean_gt * (2 - mt.__mean_gt) * n_variants / 2))
    mt = mt.unfilter_entries()

    normalized_gt = hl.or_else((mt.__gt - mt.__mean_gt) / mt.__hwe_scaled_std_dev, 0.0)
    return normalized_gt

In [6]:
# Import/generate genetic data in Hardy-Weinberg equilibrium
# to run algorithm on more-realistic data
mt = hl.balding_nichols_model(3, 1000, 10000)
norm_gt = hwe_normalize(mt.GT)
np_matrix = hl.linalg.BlockMatrix.from_entry_expr(norm_gt).to_numpy()

Initializing Hail with default parameters...
Running on Apache Spark version 2.4.1
SparkUI available at http://192.168.0.166:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.46-bf7b7f7082e1
LOGGING: writing to /Users/annamira/Documents/GitHub/hail/hail/python/hail-20200710-1505-0.2.46-bf7b7f7082e1.log
2020-07-10 15:05:49 Hail: INFO: balding_nichols_model: generating genotypes for 3 populations, 1000 samples, and 10000 variants...
2020-07-10 15:06:05 Hail: INFO: Coerced sorted dataset
2020-07-10 15:06:15 Hail: INFO: Wrote all 3 blocks of 10000 x 1000 matrix with block size 4096.


In [7]:
# Run fastPCA on genetic data

A = np.asmatrix(np_matrix)
m, n = np_matrix.shape
k = int((m + n)/4)
q = 2

U, S, V, Q, adjointQ = fastSVD(A, m, n, k, q)

# Look at differences:

(u, s, v) = np.linalg.svd(A)
simple_diff = s - S
print("subtracted difference between singular values: ", simple_diff)
print("maximum singular value difference: ", max(simple_diff))

diff = A - Q @ adjointQ @ A
norm_diff = np.linalg.norm(diff, 2)

# this method of error calculation not actually applicable?
# print("L2 norm error of ||A - QQ*A||: ", norm_diff)
# print(norm_diff < errorBound1(S, m, n, k, q))

print(norm_diff < aPosterioriError(S, m, n, k, Q, adjointQ, A))

subtracted difference between singular values:  [-3.55271368e-15  8.88178420e-15 -4.44089210e-16  4.44089210e-16
  0.00000000e+00  2.22044605e-16  2.22044605e-16 -2.22044605e-16
 -8.88178420e-16 -2.22044605e-16 -4.44089210e-16  0.00000000e+00
  0.00000000e+00 -2.22044605e-16  8.88178420e-16 -8.88178420e-16
 -2.22044605e-16 -1.11022302e-15 -4.44089210e-16  2.22044605e-16
 -8.88178420e-16 -2.22044605e-16 -2.22044605e-16 -6.66133815e-16
 -4.44089210e-16 -6.66133815e-16 -8.88178420e-16 -2.22044605e-16
 -4.44089210e-16  2.22044605e-16  0.00000000e+00  0.00000000e+00
 -6.66133815e-16  2.22044605e-16 -2.22044605e-16 -4.44089210e-16
  0.00000000e+00 -8.88178420e-16 -2.22044605e-16 -4.44089210e-16
 -4.44089210e-16  2.22044605e-16  2.22044605e-16 -4.44089210e-16
 -2.22044605e-16  8.88178420e-16  2.22044605e-16  6.66133815e-16
 -4.44089210e-16 -6.66133815e-16 -2.22044605e-16 -6.66133815e-16
  2.22044605e-16  0.00000000e+00  4.44089210e-16  4.44089210e-16
  6.66133815e-16 -2.22044605e-16  2.220446

True


In [27]:
# Basic algorithm from Banczos paper
# HAS DIMENSIONALITY ISSUES, will fail

mt2 = hl.balding_nichols_model(3, 1000, 1000)
norm_gt2 = hwe_normalize(mt2.GT)
np_matrix2 = hl.linalg.BlockMatrix.from_entry_expr(norm_gt2).to_numpy()


# again we use:
A = np.asmatrix(np_matrix2)
m, n = np_matrix2.shape
k = int((m + n)/16)
q = 2
i = 5

print("m, n: ", m, n)
print("k: ", k)

assert 2*k < m <= n

# we choose an integer l > k such that l ≤ m − k
l = int(m/2)
print("l: ", l)
assert l > k and l <= m - k

G = np.random.normal(0, 1, (l, m))
AAt = A @ A.transpose()
AAti = AAt @ AAt
for j in range(0,i):
    AAti = AAti @ AAt
R = G @ AAti @ A
assert R.shape == (l, n)

# having dimension issues here because it does not use
# the method of creating Q from the paper
(Q, S) = np.linalg.qr(R.transpose())
print("Q", Q.shape, (l, k))
print("S", S.shape, (k, n))

(Ru, Rs, Rv) = np.linalg.svd(R)
print(Ru.shape, (n, l))
print(Rs.shape, l)
print(Rv.shape, (l, l))
s_sorted = np.sort(Rs)

# print(s_sorted)
# print(reverse(s))
# assert (s == s_sorted).all()

k_largest = s[k]
assert np.linalg.norm(Q @ S - R.transpose(), 2) < k_largest

T = A @ Q
print("T", T.shape, (m, k))
assert T.shape == (m, k)
(Tu, Ts, Tw) = np.linalg.svd(T)
V = Q @ Tw



2020-07-10 17:44:46 Hail: INFO: balding_nichols_model: generating genotypes for 3 populations, 10000 samples, and 1000 variants...
2020-07-10 17:44:48 Hail: INFO: Coerced sorted dataset
2020-07-10 17:44:52 Hail: INFO: Wrote all 3 blocks of 1000 x 10000 matrix with block size 4096.


m, n:  1000 10000
k:  687


AssertionError: 

In [42]:
# Banczos algorithm

def blanczosSVD(A, m, n, k, l, q):

    assert l > k
    assert (q+1)*l <= (m - k)
    assert m <= n

    G = np.random.normal(0, 1, (l, m))
    R = G @ A
    listR = [R]
    for i in range(0, q):
        Ri = (listR[i] @ A.transpose()) @ A
        listR.append(Ri)
        R = np.concatenate((R, Ri), axis=0)

    (Q, S) = np.linalg.qr(R.transpose())
    assert(Q.shape == (n, (q+1)*l))
    assert(S.shape == ((q+1)*l, (q+1)*l))

    T = A @ Q
    assert(T.shape == (m, (q+1)*l))
    
    (Tu, Ts, Tw) = np.linalg.svd(T)
    print(Tu.shape, (m, (q+1)*l))
    assert(Ts.shape == ((q+1)*l, (q+1)*l))
    assert(Tw.shape == ((q+1)*l, (q+1)*l))
    
    
    V = Q @ Tw
    print("V shape", V.shape)
    
    bound = blanczosError(Tu, Ts, V, m, n, k, q, A, Ts[k])
    print("Satisfies Blanczos error bound equation 4.3: ", bound)

    return (Tu, Ts, V)


mt3 = hl.balding_nichols_model(3, 10000, 1000)
norm_gt3 = hwe_normalize(mt3.GT)
np_matrix3 = hl.linalg.BlockMatrix.from_entry_expr(norm_gt3).to_numpy()
print(np_matrix3.shape)

A = np.asmatrix(np_matrix3)
(m, n) = A.shape
print(A.shape)
k = 50
l = k + 12
q = 8

(blanczosU, blanczosS, blanczosV) = blanczosSVD(A, m, n, k, l, q)

2020-07-10 22:51:20 Hail: INFO: balding_nichols_model: generating genotypes for 3 populations, 10000 samples, and 1000 variants...
2020-07-10 22:51:21 Hail: INFO: Coerced sorted dataset
2020-07-10 22:51:25 Hail: INFO: Wrote all 3 blocks of 1000 x 10000 matrix with block size 4096.


(1000, 10000)
(1000, 10000)
(1000, 1000) (1000, 558)


AssertionError: 